In [ ]:
'''
Importing relevant libraries for project.
'''
import pandas as pd
import csv
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_element_located
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
'''
Importing CSV file valid postcodes as a list.
'''
POA_2016 = open('VIC_postcodes_batch3.csv')
valid_postcodes = csv.reader(POA_2016)

for row in valid_postcodes:
    valid_postcodes = row

In [ ]:
valid_postcodes

In [ ]:
'''
Creating a counter for number of iterations and an empty master dictionary for scraped IGA store data to be appended to later on.
'''
store_counter = 0
IGA_stores = { }

'''
Selenium set up and navigation to IGA 'Store locator' website.
'''
browser = webdriver.Chrome()
browser.get('https://www.iga.com.au/stores/#view=storelocator')

'''
Selenium is being used to simulate a user navigating through the IGA website and inputting postcodes into the search box.

Selenium will:
Navigate to the search box, clear text (if there is any) from the search box, input a postcode, wait for the suggestions box to load, press the down arrow
key and then enter key to select the first suggestion of the dropdown list.
      
This code is within a for loop, to iterate through the list of valid Australian postcodes.
'''
for postcode in valid_postcodes:
    
    time.sleep(5)
    
    search_box = browser.find_element_by_id("sf-location-search")
    
    search_box.clear()
    search_box.send_keys(postcode)
    search_box.click()
    
    time.sleep(5)

    search_box.send_keys(Keys.ARROW_DOWN)
    search_box.send_keys(Keys.RETURN)
    
    time.sleep(5)
    
    """
    Selenium will now scrape the store name and store address for each store in the search results. These resulting text is then cleaned
    before being stored in lists.
    """
    store_name = [ ]
    store_address = [ ]

    for store in browser.find_elements_by_xpath('.//span[@class = "sf-storename"]'):
        store_name.append(store.text)

    for address in browser.find_elements_by_xpath('.//p[@class = "sf-storeaddress"]'):
        store_address.append(address.text)

    cleaned_store_address_list = [ ]

    for store in store_address:
        cleaned_store_address_list.append(store.replace('\n' , ','))
    
    """
    Iterate through the two lists of store name and store address and add them to the master dictionary that we created earlier.
    """
    for i in range(len(store_name)):
        IGA_stores.update( {store_name[i] : cleaned_store_address_list[i] } )
    
    print(f"Stores for postcode {postcode} have been added to the master dictionary")
    
    store_counter += 1
    
    percentage_progress = float(store_counter)/len(valid_postcodes)*100 
    print(f"Progress: {round(percentage_progress,1)}%")
    
    """
    Now that all search results for this postcode have been extracted and stored, find and click on the change location button on the website, ready for 
    the next postcode to be input into the search box.
    """
    change_location = browser.find_element_by_id("sf-location-change")
    change_location.click()

In [ ]:
IGA_stores

In [ ]:
'''
Finally, take the fully populated master dictionary and output to a .csv file.
'''
(pd.DataFrame.from_dict(data = IGA_stores, orient = 'index').to_csv('scraped_VIC_IGA_stores_batch3.csv', header=False))

In [ ]:
"""
Problem postcode: 3647 (near shepparton, no IGA's closeby so no suggestion box loads)
"""